<h2>Data Preparation</h2>

In [68]:
import gc

# Force Garbage Collector to realease unreferenced memory
gc.collect()

20

In [69]:
import pandas as pd

DATASET_FILEPATH = "../_data/StructuredSmartPhonesDataset.csv"

df = pd.read_csv(DATASET_FILEPATH, header=0, sep=",")
df.head()

,Title,Brand,Model,Price,Currency,Color,MatchingCode
0,leifheit kleidersack lang farbe schwarz,NaN,NaN,14.99,EUR,schwarz,295639
1,leifheit kleidersack lang farbe schwarz,NaN,NaN,14.99,EUR,schwarz,295639
2,leifheit kleidersack lang farbe schwarz,NaN,NaN,14.99,EUR,schwarz,295639
3,leifheit kleidersack lang farbe schwarz,NaN,NaN,14.99,EUR,schwarz,295639
4,leifheit kleidersack lang farbe schwarz,NaN,NaN,14.99,EUR,schwarz,295639


In [70]:
print("Initial shape of dataset: {}".format(df.shape))

# Drop non-relevant columns for classification
df = df.drop(columns=["Title"])

# Remove rows with NULL values
df = df.dropna()

# Limit number of rows
df = df.head(10000)

print("Final shape of dataset: {}".format(df.shape))

n_unique_classes = len(set(df["MatchingCode"].values))
n_unique_rows = df.drop_duplicates().shape[0]

print("\nNumber of unique classes (MatchingCode): {}".format(n_unique_classes))
print("Number of unique rows: {}".format(n_unique_rows))

# Export processed dataset to file
df.to_csv("../_data/StructuredSmartPhonesDatasetCleaned.csv", index=False)

Initial shape of dataset: (50000, 7)
Final shape of dataset: (10000, 6)

Number of unique classes (MatchingCode): 214
Number of unique rows: 1517


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

X = df.drop(columns=["MatchingCode"])
y = df["MatchingCode"].values

# Codify string variables with OneHot encoding
enc = OneHotEncoder()
enc.fit(X)
X_encoded = enc.transform(X)

print("Shape of One-Hot-Encoded dataset: {}".format(X_encoded.shape))

# Train and test sets are stratified, i.e. they both contain the same proportion of classes than the original set
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=1, shuffle=True)

Shape of One-Hot-Encoded dataset: (10000, 1210)


<h2>K Nearest Neighbours</h2>

In [72]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import time

knn = KNeighborsClassifier()

# Perform a Grid Search to find the best values for the hyperparameters (k and weights)
# using Cross Validation with 4 stratified partitions
param_grid = {
    "n_neighbors": range(1, 11),
    "weights": ["uniform", "distance"]
}

grid_search = GridSearchCV(knn, param_grid=param_grid, cv=4)

start_time = time.time()
grid_search.fit(X_train, y_train)
finish_time = time.time()

print("Grid Search for k-NN took {} seconds.".format(round(finish_time - start_time, 3)))

# Results on test set
mean_scores = grid_search.cv_results_["mean_test_score"]
best_params = grid_search.best_params_

best_params

C:\Users\tegua\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Grid Search for k-NN took 135.514 seconds.


{'n_neighbors': 4, 'weights': 'distance'}

In [76]:
import numpy as np
from sklearn.metrics import confusion_matrix

best_k = best_params["n_neighbors"]
best_weights = best_params["weights"]

print("Best values for the k-NN hyperparameters: k = {}, weights = {}".format(best_k, best_weights))

knn = KNeighborsClassifier(n_neighbors=best_k, weights=best_weights)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

Best values for the k-NN hyperparameters: k = 4, weights = distance


In [77]:
total_ins = len(y_pred)
n_pred_ok = np.sum(y_test == y_pred)
n_pred_ko = total_ins - n_pred_ok
acc = n_pred_ok / total_ins * 100

print("Accuracy: {}".format(acc))
print("Number of instances correctly classified: {}/{}".format(n_pred_ok, total_ins))
print("Number of instances incorrectly classified: {}/{}".format(n_pred_ko, total_ins))

# inspect instances incorrecly classified (some "head", for example)


Accuracy: 92.95
Number of instances correctly classified: 1859/2000
Number of instances incorrectly classified: 141/2000


<h2>Support Vector Machines</h2>